In [9]:
import pandas as pd

In [10]:
path = 'data/CC/Validation_GCC-1.1.0-Validation.tsv'
df = pd.read_csv(path, sep='\t', names=['text', 'image'])

In [11]:
data = df.head(5)

In [12]:
len(df)

15840

In [13]:
data

,text,image
0,author : a life in photography -- in pictures,https://i.pinimg.com/736x/66/01/6c/66016c3ba27...
1,an angler fishes river on a snowy day .,http://www.standard.net/image/2015/02/04/800x_...
2,photograph of the sign being repaired by brave...,http://indianapolis-photos.funcityfinder.com/f...
3,the player staring intently at a computer scre...,http://www.abc.net.au/news/image/9066492-3x2-7...
4,globes : the green 3d person carrying in hands...,https://www.featurepics.com/StockImage/2009031...


In [ ]:
# 下载图片到本地路径 data/CC/images/validation
# 并在df加上一列相对路径 image_path
import os
import requests
from tqdm import tqdm
def download_image(url, path):
    if not os.path.exists(path):
        os.makedirs(path)
    filename = url.split('/')[-1]
    filepath = os.path.join(path, filename)
    if not os.path.exists(filepath):
        r = requests.get(url, stream=True)
        with open(filepath, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size=1024)):
                if chunk:
                    f.write(chunk)
    return filepath